<a href="https://colab.research.google.com/github/mianyumifen-bot/codePublic/blob/main/%E7%AC%AC6%E5%A4%A9sential700%E7%B1%B3%E7%BC%93%E5%86%B2%E5%8C%BA%E5%8F%AA%E4%BD%BF%E7%94%A8QA60(2025_10_26).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# -*- coding: utf-8 -*-
# 第一单元格：安装必要的库
# 这个命令会安装所有处理地理数据和与Google Earth Engine交互所需的库
# --quiet 参数可以减少安装过程中的输出信息
!pip install --quiet earthengine-api geemap rasterio rioxarray geopandas shapely pyproj

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 43.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 30.5 MB/s eta 0:00:00


In [3]:
# -*- coding: utf-8 -*-
# 第二单元格：导入、挂载硬盘与GEE授权

# 导入标准库
import os
import time
import json
import math
import datetime

# 导入地理空间处理和GEE库
import geopandas as gpd
import pandas as pd
import ee
import geemap
from shapely.geometry import mapping
# ======================================================
#  批量取消 GEE 任务的代码单元格 (已修正)
# ======================================================

import ee

# 确保 GEE 已被初始化
try:
    ee.Initialize(project="ee-mianyumifen")
except Exception as e:
    print("需要授权，请根据提示操作...")
    ee.Authenticate()
    ee.Initialize(project="ee-mianyumifen")

print("正在获取任务列表...")

# 获取所有任务的状态列表
try:
    tasks = ee.batch.Task.list()
except Exception as e:
    print(f"获取任务列表失败，请检查 GEE 授权。错误: {e}")
    tasks = [] # 赋值一个空列表以避免后续错误

# 计数器
canceled_count = 0

# 遍历所有任务
for task in tasks:
    # 检查任务状态是否为 'RUNNING' (正在运行) 或 'READY' (准备就绪，等待运行)
    if task.state == 'RUNNING' or task.state == 'READY':
        # 【已修正】从 task.config 字典中获取 'description'
        task_description = task.config.get('description', 'N/A') # 使用 .get() 更安全
        print(f"正在取消任务: {task_description} (ID: {task.id}, 状态: {task.state})")
        task.cancel() # 发送取消请求
        canceled_count += 1

if canceled_count > 0:
    print(f"\n已成功发送 {canceled_count} 个任务的取消请求。")
    print("请稍等片刻，GEE 服务器需要一些时间来处理这些请求。刷新 GEE 的 Tasks 页面可以查看状态变化。")
else:
    print("\n没有找到正在运行或等待中的任务。")


# 挂载Google Drive，这样我们就可以访问里面的文件
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# 授权并初始化Earth Engine API
# 运行时会弹出一个链接，需要你点击授权
try:
    # 填入你自己的GEE项目名称
    ee.Initialize(project="ee-mianyumifen")
    print("Google Earth Engine 已成功初始化。")
except Exception as e:
    print("需要进行GEE授权，请根据提示操作...")
    ee.Authenticate()
    ee.Initialize(project="ee-mianyumifen")
    print("Google Earth Engine 已成功授权并初始化。")

# 确认当前工作目录和Drive挂载路径
print("\n当前工作目录:", os.getcwd())
print("谷歌硬盘挂载路径: /content/drive")

需要授权，请根据提示操作...
正在获取任务列表...

没有找到正在运行或等待中的任务。
Mounted at /content/drive
Google Earth Engine 已成功初始化。

当前工作目录: /content
谷歌硬盘挂载路径: /content/drive


In [4]:
# -*- coding: utf-8 -*-
# 第三单元格：配置参数 (最终方案 - 中转站模式)

# ========== 文件夹和文件路径 (请根据你的情况修改) ==========
DRIVE_INPUT_DIR = '/content/drive/MyDrive/allPoint'

# 【已修改】定义一个单一、扁平的“中转站”文件夹，用于接收所有导出的文件
STAGING_EXPORT_FOLDER = 'S2_700_Staging_Exports'

# 【已修改】最终整理好的文件的根目录
FINAL_OUTPUT_ROOT_FOLDER = 'S2_700_Yearly_Composites'

LOG_PATH = os.path.join(DRIVE_INPUT_DIR, 's2_yearly_export_log.json')

# ========== 导出与处理控制参数 ==========
HLS_COLLECTION = 'COPERNICUS/S2_SR_HARMONIZED'
PREFERRED_BANDS = ['B4', 'B3', 'B2', 'B11']
EXPORT_SCALE = 10
BUFFER_RADIUS = 700
SLEEP_BETWEEN_START = 1

In [5]:
# -*- coding: utf-8 -*-
# 第四单元格：加载并准备点位Shapefile数据

# 构建shapefile的完整路径
roi_shp_path = os.path.join(DRIVE_INPUT_DIR, 'allPoint.shp')

# 检查文件是否存在
if not os.path.exists(roi_shp_path):
    raise FileNotFoundError(f"未在指定路径找到shapefile: {roi_shp_path}，请检查第三单元格中的路径设置。")

# 使用GeoPandas读取shapefile
rois_gdf = gpd.read_file(roi_shp_path)
print(f"成功从shapefile中加载了 {len(rois_gdf)} 个点。")
print("Shapefile中的字段名:", rois_gdf.columns.tolist())

# 自动选择 'site' 或 'name' 字段作为地点名称
if 'site' in rois_gdf.columns:
    name_field = 'site'
elif 'name' in rois_gdf.columns:
    name_field = 'name'
else:
    # 如果没找到，就自动创建一个默认的地点名字段
    name_field = 'site'
    rois_gdf[name_field] = [f"roi_{i}" for i in range(len(rois_gdf))]
    print("警告：未找到 'site' 或 'name' 字段，已自动生成默认的 'site' 字段。")

# 创建一个字典，格式为 {地点名称: 地理位置信息}
roi_dict = {str(row[name_field]): row.geometry for _, row in rois_gdf.iterrows()}
print("\nShapefile中的地点名称示例:", list(roi_dict.keys())[:10])

成功从shapefile中加载了 11 个点。
Shapefile中的字段名: ['OBJECTID_1', 'name', 'longitude', 'latitude', 'elve', 'orient', 'daihao', 'OBJECTID', 'UID', 'geometry']

Shapefile中的地点名称示例: ['norriepoint', 'cmarshhighsaltmarsh', 'bnzrichfen', 'brackishimpoundment', 'gcesapelo', 'hillslough', 'mayberry', 'northinletsaltmarsh', 'richmondbrackishmarsh', 'siwetland']


In [6]:
# -*- coding: utf-8 -*-
# 第五单元格：生成“站点-年份”处理列表

# 你可以在这里指定只处理部分站点
# 如果列表为空，则会处理shapefile中的所有站点
target_sites = [
    # 示例: 'northinletsaltmarsh', 'norriepoint' 等
]

# 如果列表为空，则自动填充所有从shapefile中读取的站点
if not target_sites:
    target_sites = list(roi_dict.keys())
    print("`target_sites` 列表为空，将处理shapefile中的所有站点。")

current_year = datetime.datetime.now().year
pairs = []  # 这个列表将用于存储所有待处理的 (站点, 年份) 组合

print("\n正在查询GEE以确定每个站点的可用年份...")
for site in target_sites:
    if site not in roi_dict:
        print(f"  警告: 目标站点 '{site}' 在shapefile中未找到，已跳过。")
        continue

    # 创建一个GEE的Geometry对象并设置缓冲区，用于检查影像是否存在
    ee_point = shapely_to_ee(roi_dict[site])
    ee_roi_for_check = ee_point.buffer(BUFFER_RADIUS)
    col = ee.ImageCollection(HLS_COLLECTION).filterBounds(ee_roi_for_check)

    # 检查该地点是否有影像
    col_size = col.size().getInfo()
    print(f"[{site}] 所有年份共找到 {col_size} 景影像。")
    if col_size == 0:
        print(f"  -> {site} 周围 {BUFFER_RADIUS}米 内未发现任何Sentinel-2影像，已跳过。")
        continue

    # 找到该地点的第一张影像的年份
    try:
        first_img = col.sort('system:time_start').first()
        first_ts = first_img.get('system:time_start').getInfo()
        first_year = datetime.datetime.utcfromtimestamp(first_ts / 1000).year
    except Exception as e:
        print(f"  -> 无法确定 {site} 的首个年份 (错误: {e})，将默认从2015年开始。")
        first_year = 2015

    # 从首个年份到当前年份，为每一年都创建一个处理任务
    for y in range(first_year, current_year + 1):
        pairs.append((site, str(y)))

print(f"\n已生成总计 {len(pairs)} 个“站点-年份”组合待处理。")
print("队列中的前40个任务:", pairs[:40])

`target_sites` 列表为空，将处理shapefile中的所有站点。

正在查询GEE以确定每个站点的可用年份...
[norriepoint] 所有年份共找到 1353 景影像。


/tmp/ipython-input-463267105.py:40: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  first_year = datetime.datetime.utcfromtimestamp(first_ts / 1000).year


[cmarshhighsaltmarsh] 所有年份共找到 1256 景影像。
[bnzrichfen] 所有年份共找到 1173 景影像。
[brackishimpoundment] 所有年份共找到 1261 景影像。
[gcesapelo] 所有年份共找到 625 景影像。
[hillslough] 所有年份共找到 679 景影像。
[mayberry] 所有年份共找到 1349 景影像。
[northinletsaltmarsh] 所有年份共找到 2518 景影像。
[richmondbrackishmarsh] 所有年份共找到 1263 景影像。
[siwetland] 所有年份共找到 1349 景影像。
[vancouversaltmarsh] 所有年份共找到 1262 景影像。

已生成总计 118 个“站点-年份”组合待处理。
队列中的前40个任务: [('norriepoint', '2015'), ('norriepoint', '2016'), ('norriepoint', '2017'), ('norriepoint', '2018'), ('norriepoint', '2019'), ('norriepoint', '2020'), ('norriepoint', '2021'), ('norriepoint', '2022'), ('norriepoint', '2023'), ('norriepoint', '2024'), ('norriepoint', '2025'), ('cmarshhighsaltmarsh', '2015'), ('cmarshhighsaltmarsh', '2016'), ('cmarshhighsaltmarsh', '2017'), ('cmarshhighsaltmarsh', '2018'), ('cmarshhighsaltmarsh', '2019'), ('cmarshhighsaltmarsh', '2020'), ('cmarshhighsaltmarsh', '2021'), ('cmarshhighsaltmarsh', '2022'), ('cmarshhighsaltmarsh', '2023'), ('cmarshhighsaltmarsh', '2024'), ('cmar

In [1]:
# -*- coding: utf-8 -*-
# 第六单元格：辅助函数与云掩膜逻辑 (仅QA60版)

def shapely_to_ee(geom):
    """将Shapely库的geometry对象转换为GEE的ee.Geometry对象。"""
    return ee.Geometry(mapping(geom))

def make_mask_function(bandnames_list):
    """
    【已修改】创建一个云掩膜函数，该函数仅使用QA60波段进行云掩膜。
    1. 如果QA60波段存在，则使用它进行掩膜。
    2. 如果QA60不存在，则不进行任何掩膜。
    """
    bandset = set(bandnames_list)
    use_qa60 = 'QA60' in bandset

    def mask_fn(img):
        img = ee.Image(img)

        # 【已修改】仅使用QA60进行掩膜
        if use_qa60:
            qa = img.select('QA60')
            # 位 10 是不透明云 (opaque cloud)
            # 位 11 是卷云 (cirrus)
            # 我们希望这两个位都是0，表示晴空
            opaque_cloud_bit = 1 << 10
            cirrus_bit = 1 << 11
            mask = qa.bitwiseAnd(opaque_cloud_bit).eq(0).And(qa.bitwiseAnd(cirrus_bit).eq(0))
            return img.updateMask(mask)

        # 如果QA60不存在，则不进行任何掩膜，返回原始影像
        return img

    return mask_fn

print("辅助函数和掩膜函数已定义 (仅QA60版)。")

辅助函数和掩膜函数已定义 (仅QA60版)。


In [7]:
# -*- coding: utf-8 -*-
# 第七单元格：核心处理函数 (最终方案 - 已添加比例因子)

def process_and_export_yearly_stack(site_name, year, point_geom, staging_folder, scale=EXPORT_SCALE, radius=BUFFER_RADIUS):
    """
    处理“站点-年份”，并将合成影像导出到指定的单一中转文件夹。
    """
    ee_point = shapely_to_ee(point_geom)
    ee_roi = ee_point.buffer(radius)

    # ... [筛选、掩膜的逻辑保持不变] ...
    start_date = f'{int(year)}-01-01'; end_date = f'{int(year)}-12-31'
    col = ee.ImageCollection(HLS_COLLECTION).filterDate(start_date, end_date).filterBounds(ee_roi)
    col_size = col.size().getInfo()
    print(f"[{site_name} {year}] 原始影像数量: {col_size}")
    if col_size == 0: return {'status':'no_images_in_year', 'site':site_name, 'year':year}
    first_img = ee.Image(col.first()); bandnames = first_img.bandNames().getInfo()
    mask_fn = make_mask_function(bandnames)
    use_bands = [b for b in PREFERRED_BANDS if b in bandnames]
    if not use_bands: use_bands = bandnames[:4]

    def prep_image(img):
        img = ee.Image(img)
        img_processed = mask_fn(img).clip(ee_roi)
        date = ee.Date(img_processed.get('system:time_start')).format('YYYYMMdd')
        tile_id = ee.String(img_processed.get('MGRS_TILE'))

        # 【关键修改】将原始整数值乘以比例因子 0.0001，转换为浮点型反射率
        # 首先用 .toFloat() 确保数据类型正确，然后乘以 scale
        chosen_bands = img_processed.select(use_bands).toFloat().multiply(0.0001)

        # 波段重命名逻辑不变
        new_names = [ee.String(b).cat('_').cat(tile_id).cat('_').cat(date) for b in use_bands]
        return chosen_bands.rename(new_names)

    prepared_col = col.map(prep_image)
    prepared_size = prepared_col.size().getInfo()
    print(f"  经过掩膜后，剩余晴空影像数量: {prepared_size}")
    if prepared_size == 0: return {'status':'no_clear_images', 'site':site_name, 'year':year}

    stacked_image = prepared_col.toBands()

    # 导出逻辑不变，仍然指向中转文件夹
    export_name = f"{site_name}_{year}"
    task = ee.batch.Export.image.toDrive(
        image=stacked_image,
        description=export_name,
        folder=staging_folder,
        fileNamePrefix=export_name,
        region=ee_roi,
        scale=scale,
        maxPixels=1e13
    )
    task.start()

    print(f"  -> 已提交任务: {export_name}.tif -> 中转文件夹: {staging_folder}")
    return {'status':'export_started', 'site':site_name, 'year':year, 'task_id': task.id}

print("核心处理与导出函数已定义 (最终方案 - 已添加比例因子)。")

核心处理与导出函数已定义 (最终方案 - 已添加比例因子)。


In [8]:
# -*- coding: utf-8 -*-
# 第八单元格：主循环 (最终方案 - 导出到中转站)

results = []
started_tasks_count = 0

for idx, (site, year) in enumerate(pairs, start=1):
    print(f"\n[{idx}/{len(pairs)}] 开始处理： {site} - {year}")
    if site not in roi_dict:
        results.append({'site':site, 'year':year, 'status':'site_not_found'}); continue
    try:
        # 【已修改】调用函数时传入中转站文件夹名
        res = process_and_export_yearly_stack(
            site_name=site,
            year=year,
            point_geom=roi_dict[site],
            staging_folder=STAGING_EXPORT_FOLDER, # 使用第3单元格定义的中转站变量
            scale=EXPORT_SCALE,
            radius=BUFFER_RADIUS
        )
        results.append(res)
        if res.get('status') == 'export_started': started_tasks_count += 1
    except Exception as e:
        results.append({'site':site, 'year':year, 'status':'fatal_error', 'error_msg': str(e)})

with open(LOG_PATH, 'w', encoding='utf-8') as f:
    json.dump(results, f, ensure_ascii=False, indent=4)

print("\n========================================================")
print("所有导出任务已提交到中转文件夹。")
print(f"本次运行共提交了 {started_tasks_count} 个导出任务。")
print("请等待所有任务在GEE Tasks页面完成后，再运行第九单元格的整理脚本。")
print("========================================================")


[1/118] 开始处理： norriepoint - 2015
[norriepoint 2015] 原始影像数量: 13
  经过掩膜后，剩余晴空影像数量: 13
  -> 已提交任务: norriepoint_2015.tif -> 中转文件夹: S2_700_Staging_Exports

[2/118] 开始处理： norriepoint - 2016
[norriepoint 2016] 原始影像数量: 67
  经过掩膜后，剩余晴空影像数量: 67
  -> 已提交任务: norriepoint_2016.tif -> 中转文件夹: S2_700_Staging_Exports

[3/118] 开始处理： norriepoint - 2017
[norriepoint 2017] 原始影像数量: 123
  经过掩膜后，剩余晴空影像数量: 123
  -> 已提交任务: norriepoint_2017.tif -> 中转文件夹: S2_700_Staging_Exports

[4/118] 开始处理： norriepoint - 2018
[norriepoint 2018] 原始影像数量: 142
  经过掩膜后，剩余晴空影像数量: 142
  -> 已提交任务: norriepoint_2018.tif -> 中转文件夹: S2_700_Staging_Exports

[5/118] 开始处理： norriepoint - 2019
[norriepoint 2019] 原始影像数量: 144
  经过掩膜后，剩余晴空影像数量: 144
  -> 已提交任务: norriepoint_2019.tif -> 中转文件夹: S2_700_Staging_Exports

[6/118] 开始处理： norriepoint - 2020
[norriepoint 2020] 原始影像数量: 145
  经过掩膜后，剩余晴空影像数量: 145
  -> 已提交任务: norriepoint_2020.tif -> 中转文件夹: S2_700_Staging_Exports

[7/118] 开始处理： norriepoint - 2021
[norriepoint 2021] 原始影像数量: 144
  经过掩膜后，剩余晴空影像数量: 144